In [8]:
from alert import alert 

date = '2024-06-18'

json_r = alert(date)
print(json_r)

{
    "type": "bubble",
    "size": "giga",
    "body": {
        "type": "box",
        "layout": "vertical",
        "contents": [
            {
                "type": "text",
                "text": "REPORT FOR ",
                "weight": "bold",
                "color": "#1DB446",
                "size": "sm"
            },
            {
                "type": "text",
                "text": "Brown Store",
                "weight": "bold",
                "size": "xxl",
                "margin": "md",
                "contents": [
                    {
                        "type": "span",
                        "text": "2024-06-18"
                    }
                ]
            },
            {
                "type": "separator",
                "margin": "xxl"
            },
            {
                "type": "text",
                "text": "COMMENT",
                "weight": "bold",
                "color": "#1DB446",
                "size": "sm"
            },
 

In [1]:
import os
import sys

file_dir = os.getcwd()
chatbot_dir = os.path.join(file_dir, '..')
bloomberg_dir = os.path.join(chatbot_dir, '..')
dirs = [chatbot_dir, bloomberg_dir, file_dir]

sys.path.extend(d for d in dirs if d not in sys.path)

import polars as pl 
import pandas as pd 
import json 

data_dir = os.path.join(bloomberg_dir,'data')

from Flex import Flex
import warnings 
from helpers.Database import Database

warnings.filterwarnings('ignore')

def get_alert(date:str = None):
    db = Database()
    df_raw = db.get_data('bloomberg')

    if not date:
        yesterday = (pd.to_datetime('today') - pd.DateOffset(days=1)).date()
    else :
        yesterday = pd.to_datetime(date).date()

    df_raw['create_at'] = pd.to_datetime(df_raw['create_at']).dt.date
    df_raw['date'] = pd.to_datetime(df_raw['date']).dt.date
    
    df = df_raw.copy()
    df = df[df['date'] <= yesterday]

    df.drop_duplicates(subset=['symbol', 'broker', 'comment', 'target'], keep='first', inplace=True)

    df = df.sort_values(by=['broker', 'symbol', 'date'], ascending=True)

    # -------------------------------- New Listed -------------------------------- #
    df['count'] = df.groupby(['broker', 'symbol'])['symbol'].transform('count')

    new_listed = df[(df['count'] == 1) & (df['date'] == yesterday)]
    new_listed = new_listed[['broker', 'symbol', 'target', 'comment']].reset_index(drop=True)
    new_listed.sort_values('symbol',ascending=True, inplace=True)
    # ---------------------------------- Changes --------------------------------- #
    df['previous_target'] = df.groupby(['broker', 'symbol'])['target'].shift(1)
    df['previous_comment'] = df.groupby(['broker', 'symbol'])['comment'].shift(1)
    df['previous_date'] = df.groupby(['broker', 'symbol'])['date'].shift(1)
    # ----------------------- Change that occured yesterday ---------------------- #
    df.dropna(subset=['previous_target', 'previous_comment'], inplace=True, how ='all')
    yesterday_changes = df[df['date'] == yesterday].copy()
    # ------------------------------ Target Changes ------------------------------ #
    target_changes = yesterday_changes[yesterday_changes['target'] != yesterday_changes['previous_target']]
    target_changes = target_changes[['broker', 'target', 'previous_target', 'symbol', 'date', 'previous_date']]
    target_changes['diff'] = target_changes['target'] - target_changes['previous_target']

    target_changes = target_changes.astype({'target': 'str'})
    target_changes.loc[:,'target'] = target_changes['target'] + ' (' + round(target_changes['diff'],2).astype(str) + ')'
    target_used = target_changes[['broker', 'symbol', 'previous_target', 'target']].reset_index(drop=True)
    target_used.columns = ['broker', 'symbol', 'from', 'to']

    # ------------------------------ Comment Changes ----------------------------- #
    comment_changes = yesterday_changes[yesterday_changes['comment'] != yesterday_changes['previous_comment']]
    comment_changes = comment_changes[['broker', 'comment', 'previous_comment', 'symbol', 'date', 'previous_date']]

    comment_used = comment_changes[['broker', 'symbol', 'previous_comment', 'comment']].reset_index(drop=True)
    comment_used.columns = ['broker', 'symbol', 'from', 'to']
    
    res = {
        'new_listed': new_listed.head(10),
        'target': target_used,
        'comment': comment_used,
        'date': yesterday.strftime('%Y-%m-%d')
    }
    
    return res

In [2]:
res = get_alert('2024-06-18')

In [17]:
res['comment'].reset_index(drop=True)

,broker,symbol,from,to
0,Bualuang Securities,HANA,hold,trading buy
1,KGI Securities Co Ltd,BCP,neutral,outperform
2,Krungsri Group,BEC,sell,neutral
3,Phillip Securities,STA,hold,neutral
4,Tisco Securities,KCE,hold,buy
5,UOB KayHian (Equity),OSP,hold,buy


In [3]:
import os
import sys

file_dir = os.getcwd()
chatbot_dir = os.path.join(file_dir, '..')
bloomberg_dir = os.path.join(chatbot_dir, '..')
dirs = [chatbot_dir, bloomberg_dir, file_dir]

sys.path.extend(d for d in dirs if d not in sys.path)

import polars as pl 
import pandas as pd 
import json 

data_dir = os.path.join(bloomberg_dir,'data')

from Flex import Flex
from helpers.Database import Database
import warnings 

warnings.filterwarnings('ignore')

sym = 'KBANK'
flex = Flex(sym)

broker_list = pl.read_csv(os.path.join(data_dir, 'broker.csv')).to_pandas()
type_dict = broker_list[['broker', 'type']].set_index('broker').to_dict()['type']

In [4]:
db = Database()
year_df = db.query(f"SELECT * FROM bloomberg WHERE symbol = '{sym}' AND YEAR(date) = YEAR(CURDATE())")
year_df['date'] = pd.to_datetime(year_df['date'])


year_df['type'] = year_df['broker'].map(type_dict)

# ---------------------------------- Average --------------------------------- #
avg_sep = year_df.pivot_table(index='type', values ='target', aggfunc='mean').rename(columns = {'target': "Average Target Price"}).round(2)
avg_sep_local = avg_sep.loc[['local']].T.reset_index().T.reset_index(drop=True)
avg_sep_foreign = avg_sep.loc[['foreign']].T.reset_index().T.reset_index(drop=True)
# ----------------------------------- Mode ----------------------------------- #
mode_sep = year_df.pivot_table(index=['type','comment'] , aggfunc='count')['analyst'].reset_index()\
            .groupby('type').apply(lambda x: x.sort_values('analyst', ascending=False).head(1))
            
mode_sep['Most Common Comment'] = mode_sep['comment'] + ' (' + mode_sep['analyst'].astype(str) + ')'
mode_sep_foreign = mode_sep.loc[['foreign'], ['Most Common Comment']].T.reset_index().T.reset_index(drop=True).T
mode_sep_local = mode_sep.loc[['local'], ['Most Common Comment']].T.reset_index().T.reset_index(drop=True).T

In [72]:
# ------------------------------- IM Component ------------------------------- #
im = pl.read_csv(os.path.join(data_dir, 'im.csv')).to_pandas()
im_df = im[im['symbol'] == sym]

im_df.drop(columns=['symbol'], inplace=True)
im_df.columns = ['ขั้นต่ำ', 'Leverage', 'ใช้เงิน']

im_df['ขั้นต่ำ'] = im_df['ขั้นต่ำ'].apply(lambda x: f'{x:,.0f}')
im_df['ใช้เงิน'] = im_df['ใช้เงิน'].apply(lambda x: f'{x:,.0f}')
im_df['Leverage'] = im_df['Leverage'].apply(lambda x: f'{x}')
im_df = im_df.T.reset_index()
im_df = im_df.T.reset_index(drop=True).T

In [76]:
im_df[2] = ['สัญญา', 'เท่า', 'บาท']

In [77]:
im_df

,0,1,2
0,ขั้นต่ำ,20,สัญญา
1,Leverage,19.07,เท่า
2,ใช้เงิน,"137,900",บาท
